Obtain statistics on BMLP_GPU runtime

In [1]:
import cupy as cp 
import numpy as np
import pygraphblas as gb

# Check if connected to GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU: need to configure runtime type to an environment with an accessible GPU')
else:
    print('Connected to a GPU - GPU info summary: \n\n' + gpu_info)

Connected to a GPU - GPU info summary: 

Sat Feb  8 17:18:05 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.04              Driver Version: 536.23       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  | 00000000:01:00.0  On |                  N/A |
| 32%   27C    P8               7W / 160W |    787MiB /  8188MiB |     27%      Default |
|                                         |                      |                  N/A |
+--------------------------

In [2]:
# Use python wrapper of GraphBLAS on GPU (BLAS - Basic Linear Algebra Subprograms)
# GraphBLAS supports graph operations via linear algebraic methods (e.g. matrix multiplication) over various semirings

# GraphBLAS version of BMLP-RMS algorithm which performs repeated matrix squaring
from bmlp.core.matrix import *

In [3]:
# Simple graph example 1
# Create edge list (directed graph) and the number of nodes (matrix dimension)
edges = [(0, 1), (1, 2), (2, 3), (3, 4)]
num_nodes = 5

In [4]:
# Create a square adjacency matrix using BOOL type
R1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

# Insert edges into the adjacency matrix
for src, dst in edges:
    R1[src, dst] = True
print('R1 = \n'+ str(R1) + '\n')
        
# Print the transitive closure matrix
print('closure R2* = \n' + str(RMS(R1, print_matrix=True)) + '\n')

R1 = 
      0  1  2  3  4
  0|     t         |  0
  1|        t      |  1
  2|           t   |  2
  3|              t|  3
  4|               |  4
      0  1  2  3  4

R = R2 + I = 
      0  1  2  3  4
  0|  t  t         |  0
  1|     t  t      |  1
  2|        t  t   |  2
  3|           t  t|  3
  4|              t|  4
      0  1  2  3  4

fixpoint = 
      0  1  2  3  4
  0|  t  t  t      |  0
  1|     t  t  t   |  1
  2|        t  t  t|  2
  3|           t  t|  3
  4|              t|  4
      0  1  2  3  4

fixpoint = 
      0  1  2  3  4
  0|  t  t  t  t  t|  0
  1|     t  t  t  t|  1
  2|        t  t  t|  2
  3|           t  t|  3
  4|              t|  4
      0  1  2  3  4

fixpoint = 
      0  1  2  3  4
  0|  t  t  t  t  t|  0
  1|     t  t  t  t|  1
  2|        t  t  t|  2
  3|           t  t|  3
  4|              t|  4
      0  1  2  3  4

R0* = 
      0  1  2  3  4
  0|     t  t  t  t|  0
  1|        t  t  t|  1
  2|           t  t|  2
  3|              t|  3
  4|            

In [5]:
# Simple graph example 2
# Create edge list (directed graph) and the number of nodes (matrix dimension)
edges = [(0, 1), (1, 2), (2, 3), (3, 0), (4, 4)]
num_nodes = 5

In [6]:
# Create a square adjacency matrix using BOOL type
R1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

# Insert edges into the adjacency matrix
for src, dst in edges:
    R1[src, dst] = True
print('R1 = \n'+ str(R1) + '\n')
        
# Print the transitive closure matrix
print('closure R2* = \n' + str(RMS(R1)) + '\n')

R1 = 
      0  1  2  3  4
  0|     t         |  0
  1|        t      |  1
  2|           t   |  2
  3|  t            |  3
  4|              t|  4
      0  1  2  3  4

closure R2* = 
      0  1  2  3  4
  0|  t  t  t  t   |  0
  1|  t  t  t  t   |  1
  2|  t  t  t  t   |  2
  3|  t  t  t  t   |  3
  4|              t|  4
      0  1  2  3  4



In [7]:
# Create a vector to represent a query
V = gb.Vector.sparse(gb.BOOL, num_nodes)

# query the reachability of node 3
V[3] = True

print('closure V* = \n' + str(SMP(V,R1)) + '\n')

closure V* = 
0| t
1| t
2| t
3| t
4|



In [8]:
R1 = gb.Matrix.sparse(gb.BOOL, 3, 3)
R1[0,0] = True
R1[0,1] = True
R1[1,2] = True
# R1 = R1.apply(gb.types.BOOL.LNOT)
print(R1)

print(negate(R1))


      0  1  2
  0|  t  t   |  0
  1|        t|  1
  2|         |  2
      0  1  2
      0  1  2
  0|        t|  0
  1|  t  t   |  1
  2|  t  t  t|  2
      0  1  2


In [9]:
import random, time

# Complex graph
num_nodes = 5000
all_edges = [(i, j) for i in range(num_nodes) for j in range(num_nodes)]
print('No. edges: ' + str(len(all_edges)))

No. edges: 25000000


In [10]:
# num_reps = 10
# p = 0.5
# total_time = []

# # repeat num_reps times
# for i in range(num_reps):

#     # sample edges with edge probability < p
#     sampled_edges = list(filter(lambda _: random.random() < p, all_edges))

#     # Create a square adjacency matrix using BOOL type
#     empty_matrix = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
#     R1 = empty_matrix

#     # Insert edges into the adjacency matrix
#     for src, dst in sampled_edges:
#         R1[src, dst] = True
#     R1.to_binfile("large")

#     # Run and time the BMLP-RMS module
#     start_time = time.time()
#     RMS(R1)
#     end_time = time.time()

#     total_time.append(end_time - start_time)
#     print('Wall time: ' + str(end_time - start_time))

# print('Mean wall time: ' + str(np.mean(total_time)))
# print('Wall time sterr: ' + str(np.std(total_time) / np.sqrt(len(total_time))))


In [11]:
num_reps = 10
p = 0.5
total_time = []

# repeat num_reps times
for i in range(num_reps):

    # sample edges with edge probability < p
    sampled_edges = list(filter(lambda _: random.random() < p, all_edges))

    # Create a square adjacency matrix using BOOL type
    empty_matrix = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
    R1 = empty_matrix

    # Insert edges into the adjacency matrix
    for src, dst in sampled_edges:
        R1[src, dst] = True
        
    # Create a vector to represent a query
    V = gb.Vector.sparse(gb.BOOL, num_nodes)

    # query the reachability of node 3
    V[1] = True

    # Run and time the BMLP-SMP module
    start_time = time.time()
    SMP(V,R1)
    end_time = time.time()

    total_time.append(end_time - start_time)
    print('Wall time: ' + str(end_time - start_time))

print('Mean wall time: ' + str(np.mean(total_time)))
print('Wall time sterr: ' + str(np.std(total_time) / np.sqrt(len(total_time))))

Wall time: 0.05126166343688965
Wall time: 0.0387113094329834
Wall time: 0.030705928802490234
Wall time: 0.02973771095275879
Wall time: 0.0309598445892334
Wall time: 0.031252384185791016
Wall time: 0.03593277931213379
Wall time: 0.06989812850952148
Wall time: 0.0631411075592041
Wall time: 0.03707146644592285
Mean wall time: 0.04186723232269287
Wall time sterr: 0.0043567043620277935
